In [2]:
import re
import networkx as nx
import json
from scipy.stats.stats import pearsonr
import numpy as np
import seaborn as sns
import pandas as pd
import datetime
import sys
import multiprocessing
from glob import glob
from collections import Counter
import gensim
from scipy import spatial
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
import matplotlib.pyplot as plt
# plt.rc('font',family='Times New Roman')
# from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import statsmodels.formula.api as smf
from scipy.stats import chisquare
from scipy import stats
pd.set_option('display.max_columns',None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1000)

### job_rec

In [3]:
import pdfminer #python包，时间短

In [4]:
! pdf2txt.py -o ./CV1.txt ./CV.pdf

In [5]:
with open('skill_emb.json', 'r') as json_file:
    skill_emb = json.load(json_file)

In [6]:
with open("./hubstaff_job_descriptions.json") as fp: # complete
    job_des = json.load(fp)

In [7]:
skills=[i['skills'] for i in job_des]

In [8]:
def skill(x):
    a=' '.join(x)
    a=a.split(' ')
    b=[]
    for i in a:
        if i in skill_emb:
            b.append(skill_emb[i])
    if b==[]:
        return [0]*300
    else:
        return list(np.mean(b,axis=0))

In [9]:
skill_dimen=[skill([j.lower() for j in i]) for i in skills] #计算所有工作的embedding

In [10]:
with open('./CV1.txt') as f:
    text=f.readlines()

text=[re.sub(r'[\n\r\t "]'," ",i) for i in text]
text=[i.lower() for i in ''.join(text).split(' ') if i!='']

In [11]:
my_skill=skill(list(set([i for i in text if i in skill_emb.keys()])))

In [12]:
set([i for i in text if i in skill_emb.keys()]) #从简历里面提取出在技能字典的词

{'company',
 'computer',
 'data',
 'digital',
 'education',
 'female',
 'information',
 'journalism',
 'online',
 'psychology',
 'python',
 'research',
 'science',
 'spss',
 'statistics',
 'story',
 'word'}

In [13]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

In [14]:
skill_dimen=[normalize(i) for i in skill_dimen]
a=np.array(skill_dimen).dot(np.array(normalize(my_skill))) #计算简历和所有技能的相似性

In [15]:
topn=5
[job_des[i] for i,j in sorted(enumerate(a),key=lambda x: x[1],reverse=True)[:topn]]

[{'name': 'Virtual Assistant',
  'pricing_model': 'hourly',
  'company': ' Mealheroes',
  'company_url': 'http://www.mealheroes.de',
  'location': ' Berlin, Berlin, Germany',
  'type': ' Remote job',
  'created_at': ' Jun  1',
  'description': ['We are a young food startup.',
   'Currently, we search for business tasks and for private tasks someone who can help us with Research, Word Formatting, Data Entries, and Power Point presentations.'],
  'skills': ['Presentation Design',
   'Research',
   'Word',
   'Online Research',
   'Data Entry'],
  'availability': ['Hourly contract'],
  'experience_levels': [],
  'languages': [],
  'pay_rate': '$3/hr'},
 {'name': 'Predictive Analytics for SAP PAL',
  'pricing_model': 'hourly',
  'company': ' KB Solutions',
  'company_url': 'http://www.kbsapsolutions.com',
  'location': ' Tel Aviv, Tel Aviv, Israel',
  'type': ' Remote job',
  'created_at': ' May 20',
  'description': 'Extensive knowledge in predictive analytics for SAP PAL. We need to form

In [16]:
到这一步就结束推荐！底下是可视化，还没有弄好

SyntaxError: invalid character in identifier (<ipython-input-16-a2351460f624>, line 1)

### skill_tree

In [ ]:
with open('skill_emb.json', 'r') as json_file:
    skill_emb = json.load(json_file)

In [ ]:
len(skill_emb)

In [ ]:
skill_key=[i for i in skill_emb]

In [ ]:
skill_value=[skill_emb[i] for i in skill_emb]

In [ ]:
a=np.array(skill_value).dot(np.array(skill_value).T)

In [ ]:
W=nx.Graph()
for s in range(len(skill)):
    m=skill_key[s]
    n=[skill_key[i] for i,j in sorted(enumerate(a[s]),key=lambda x: x[1],reverse=True)[1:2]][0]
    if m!=n:
        W.add_edge(m,n,weight=1)

In [ ]:
W['management']

In [ ]:
W['airbnb management']

In [ ]:
pos = nx.spring_layout(W)
T = W.degree(weight='weight')

In [ ]:
with open('./killWeb_LinkedIn_Pos.txt','w') as f:
    for i in pos:
        xi,yi=pos[i]
        f.write(i+'\t'+str(xi)+'\t'+str(yi)+'\n')

In [ ]:
pos={}
with open('./killWeb_LinkedIn_Pos.txt','r') as f:
    for line in f:
        i,x,y=line.strip().split('\t')
        pos[i]=[float(x),float(y)]

In [ ]:
fig = plt.figure(figsize=(20, 10),facecolor='white')
ax = fig.add_subplot(121)
size=np.array([len(W[i]) for i in list(pos.keys())])
x,y = np.array(list(pos.values())).T
plt.scatter(x,y,s=size,c='gray',edgecolor='gray',alpha=1)
for i in pos:
    if len(W[i])>5:
        x,y=pos[i]
        plt.text(x,y,i)
for i,j in W.edges():
    xi,yi=pos[i]
    xj,yj=pos[j]
    w=W[i][j]['weight']
    plt.plot([xi,xj],[yi,yj],color='gray',linewidth=1,alpha=1)
# plt.savefig('./socname_space.png',dpi=300)